In [1]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import sys, gc, warnings, random, math, time, datetime, os
START_DATE = datetime.datetime.strptime('2017-11-30', '%Y-%m-%d')
from tqdm import tqdm_notebook
from sklearn import preprocessing, metrics
from sklearn.model_selection import train_test_split, KFold, GroupKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
import xgboost as xgb
import lightgbm as lgb

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

from utils import *

SEED = 42
seed_everything(SEED)

In [2]:
train = pd.read_pickle('../input/reduced/train_df_features.pkl')
test = pd.read_pickle('../input/reduced/test_df_features.pkl')
train_y = pd.read_pickle('../input/reduced/train_y.pkl')

In [3]:
print('training data size is ', train.shape)
print('training label size is', train_y.shape)
print('testing data size is', test.shape)

training data size is  (590540, 522)
training label size is (590540,)
testing data size is (506691, 522)


In [9]:
product_list = ['W', 'H', 'C', 'S', 'R']
train_sepa = {}

In [10]:
for i in product_list:
    print( train[train['ProductCD'] == i].shape)
    train_sepa[i] = train[train['ProductCD'] == i]

(439670, 522)
(33024, 522)
(68519, 522)
(11628, 522)
(37699, 522)


In [13]:
rm_cols = [
    'TransactionID', 'TransactionDT', 'isFraud', 'ProductCD', 'uid', 'uid2',
    'uid3', 'DT', 'DT_month', 'DT_week_year', 'DT_day_year', 'DT_hour',
    'DT_day_week', 'DT_day', 'DT_D_total', 'DT_W_total', 'DT_M_total', 'id_30',
    'id_31', 'id_33'
]

In [59]:
########################### Features elimination 
from scipy.stats import ks_2samp
features_check = []
columns_to_check = set(list(train)).difference(rm_cols)
for i in columns_to_check:
    features_check.append(ks_2samp(train_y, train[i])[1])

features_check = pd.Series(features_check, index=columns_to_check).sort_values() 
features_discard = list(features_check[features_check == 0].index)
print(features_discard)

# We will reset this list for now,
# Good droping will be in other kernels
# with better checking
# features_discard = [] 

# Final features list
features_columns = [col for col in list(train) if col not in rm_cols + features_discard]

['C14', 'V79', 'D12', 'id_09', 'V295', 'V312', 'V149', 'D14', 'C2', 'V308', 'V261', 'V16', 'V55', 'V263', 'V277', 'V282', 'V156', 'C10', 'V132', 'V21', 'V201', 'V262', 'id_01', 'C8_fq_enc', 'card5', 'id_16', 'V81', 'D7_fq_enc', 'R_emaildomain', 'card3', 'V130', 'id_38', 'M4', 'uid_fq_enc', 'D1', 'uid2_TransactionAmt_std', 'id_17', 'V240', 'V272', 'M3', 'V58', 'V37', 'DeviceType', 'V5', 'V251', 'V138', 'V13', 'card3_fq_enc', 'V329', 'M7', 'C6', 'V226', 'V144', 'V217', 'M1', 'V60', 'V163', 'V208', 'V207', 'V285', 'V237', 'C5_fq_enc', 'V328', 'V319', 'V54', 'V131', 'V40', 'V125', 'V236', 'V256', 'V187', 'id_30_version_fq_enc', 'V105', 'addr1_fq_enc', 'V76', 'V172', 'C12_fq_enc', 'V48', 'V218', 'V22', 'D4', 'V170', 'V128', 'V327', 'V135', 'V6', 'C13', 'V41', 'id_05', 'V274', 'TransactionAmt_check', 'V270', 'V174', 'V73', 'V209', 'V253', 'V18', 'V97', 'V264', 'V318', 'V152', 'uid_DT_month', 'V332', 'V49', 'uid_DT_day_year', 'V230', 'V334', 'V313', 'V139', 'M8', 'V42', 'V62', 'card4', 'V133'